In [69]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [70]:
#accedemos a drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
#lectura del dataset
data = pd.read_csv("/content/drive/MyDrive/USFX/SIS420-IA/datasets/stats_BVG_latest.csv")

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Emisores            684 non-null    object 
 1   Valor Nominal       684 non-null    float64
 2   Último Precio       684 non-null    float64
 3   P/U  Veces          684 non-null    object 
 4   P/VL                684 non-null    float64
 5   Capitalización      684 non-null    int64  
 6   D/P  Yield          684 non-null    object 
 7   Presencia Bursátil  684 non-null    object 
 8   Índice de Rotación  684 non-null    float64
 9   Líquidez            684 non-null    object 
 10  Fecha               684 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 58.9+ KB


In [74]:
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [75]:
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Emisores            684 non-null    int64  
 1   Valor Nominal       684 non-null    float64
 2   Último Precio       684 non-null    float64
 3   P/U  Veces          684 non-null    int64  
 4   P/VL                684 non-null    float64
 5   Capitalización      684 non-null    int64  
 6   D/P  Yield          684 non-null    int64  
 7   Presencia Bursátil  684 non-null    int64  
 8   Índice de Rotación  684 non-null    float64
 9   Líquidez            684 non-null    int64  
 10  Fecha               684 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 58.9 KB


In [78]:
# Seleccionar las características y la variable objetivo
#features= caracteristicas y target=objetivo
features = data[['Valor Nominal', 'P/U  Veces', 'P/VL', 'Capitalización', 'D/P  Yield', 'Índice de Rotación', 'Líquidez']]
target = data['Último Precio']

In [79]:
# Normalizar los datos
#escala las características de tus datos utilizando la transformación de escala mínima y
#máxima, lo que resulta en características que están en el rango de 0 a 1. Esto es útil
#para asegurarse de que todas las características tengan la misma escala
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [80]:
# Convertir los datos en tensores de PyTorch para tener compatibilidad y  calculos eficientes
#se coloca los datos ya normalizados de las caracteristicas
features_tensor = torch.tensor(features_scaled, dtype=torch.float32)
target_tensor = torch.tensor(target.values, dtype=torch.float32).view(-1, 1)


In [88]:
# Definir el modelo de regresión lineal múltiple
#La clase LinearRegression hereda de la clase nn.Module,
# que es una clase base proporcionada por PyTorch para definir modelos de aprendizaje
#automático. Al heredar de nn.Module, se obtienen las funcionalidades y características
#necesarias para construir y entrenar un modelo.
class LinearRegression(nn.Module):
    def __init__(self, input_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1) #Se crea una sola capa lineal

    def forward(self, x):
        return self.linear(x)

# Definir hiperparámetros
input_size = features_tensor.shape[1]
learning_rate = 0.1 #tasa de aprendizaje
num_epochs = 10000 #numero de epocas del modelo

# Crear el modelo
model = LinearRegression(input_size)

In [89]:
# Definir la función de pérdida  y el optimizador
criterion = nn.MSELoss() #función de pérdida de error cuadrático medio
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #representa el descenso de gradiente estocástico

In [90]:
# Entrenamiento del modelo
for epoch in range(num_epochs):
    # Calcular la salida del modelo
    outputs = model(features_tensor)

    # Calcular la pérdida
    loss = criterion(outputs, target_tensor) #se calcula la perdida

    # Retropropagación y optimización para ajustaar los pesos
    optimizer.zero_grad()
    loss.backward() #se calcula los gradientes de los paramettros
    optimizer.step() #se actualiza los pesos

    # Imprimir la información del entrenamiento
    if (epoch+1) % 1000 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1000, Loss: 154.3954620361328
Epoch: 2000, Loss: 154.27084350585938
Epoch: 3000, Loss: 154.2705078125
Epoch: 4000, Loss: 154.2705078125
Epoch: 5000, Loss: 154.27052307128906
Epoch: 6000, Loss: 154.27052307128906
Epoch: 7000, Loss: 154.27052307128906
Epoch: 8000, Loss: 154.27052307128906
Epoch: 9000, Loss: 154.27052307128906
Epoch: 10000, Loss: 154.27052307128906


In [91]:
#Después de obtener los pesos y el sesgo, puedes utilizarlos para realizar
#predicciones en nuevos datos o para realizar análisis adicionales de tu modelo.
# Obtener los coeficientes y el sesgo del modelo
weights = model.linear.weight.data
bias = model.linear.bias.data

In [92]:
# Imprimir los coeficientes y el sesgo
print('Coeficientes:', weights)
print('Sesgo:', bias) #se utiliza para ajustar la línea de regresión y determinar su posición vertical

Coeficientes: tensor([[ 74.8456, -18.0099,  41.4959,   9.4948,  -7.8573,  -1.9375,   0.2591]])
Sesgo: tensor([14.6787])
